In [ ]:
pip install ase

In [ ]:
pip install smact

In [ ]:
pip install multiprocess

In [286]:
import matplotlib.pyplot as plt
import multiprocess
import numpy as np
import pandas as pd
from pymatgen.core import Composition

import smact

from smact import Species, screening
from smact.oxidation_states import Oxidation_state_probability_finder

In [288]:
# Importing shannon radius machine learning data in angstrom
import json

with open("shannon_ml.json") as f:
    shannon_data = json.load(f)

In [289]:
# Importing list of text compounds from csv

import csv
import ast  # Safely parse strings representing Python an_ox_check structures

def convert_value(value):
    """Convert values to int, float, tuple, or keep as original string."""
    value = value.strip()  # Remove extra spaces

    try:
        # Attempt to parse tuple, list, int, or float
        parsed_value = ast.literal_eval(value)
        
        # Ensure that parsed values like lists/tuples are kept as they are
        if isinstance(parsed_value, (int, float, list, tuple)):
            return parsed_value
        else:
            return value  # If it's another type, return as string
    except (ValueError, SyntaxError):
        return value  # Return as string if it can't be parsed

# Read CSV and properly restore an_ox_check types
tol_oct = []
with open('Anion and stoich check 2.csv', mode='r', newline='') as file:
    reader = csv.reader(file)
    header = next(reader)  # Read header
    tol_oct = [[convert_value(value) for value in row] for row in reader]  # Convert each value

# Output to confirm correct conversion
print(header)
print((tol_oct[0]))  



print(len(tol_oct))



['Element Combination', 'Oxidation States', 'Stoichiometry']
[('Li', 'B', 'Si'), (1, -1, 4), (2, 6, 1)]
66709


In [290]:
# Moves the positions of the variables such that they are in the 216 configuration

def move(list,position):
    global tol_oct
    box = [None,None,None]
    
    for i in range(len(list)):
        box[position[i]] = list[i]
    return box

q = ["-1", "-2", "-3", "-4", "-5","-6","-7","-8"]


count = 0


for i in tol_oct:
    position = []    
    #print(i)
    for j in range(3):

        # Checks the oxidation state of the ions and marks the one with a -1 ox state as anion
        if i[1][j]<0: 
            position.append(2)

        else:
            # Checks the stoichiometry to determine cation a and b
            if i[2][j]==2:
                position.append(0)
            elif i[2][j]==1:
                position.append(1)
    #i.append([a,b,x])
    #print(position)

    i[0] = move(i[0],position)
    i[1] = move(i[1],position)
    i[2] = move(i[2],position)



print(tol_oct[200:220])
print(len(tol_oct))

[[['Li', 'V', 'P'], [1, 4, -1], [2, 1, 6]], [['Li', 'Cr', 'P'], [1, 4, -1], [2, 1, 6]], [['Li', 'Mn', 'P'], [1, 4, -1], [2, 1, 6]], [['Li', 'Fe', 'P'], [1, 4, -1], [2, 1, 6]], [['Li', 'Co', 'P'], [1, 4, -1], [2, 1, 6]], [['Li', 'Ni', 'P'], [1, 4, -1], [2, 1, 6]], [['Li', 'Ga', 'P'], [1, 4, -1], [2, 1, 6]], [['Li', 'Ge', 'P'], [1, 4, -1], [2, 1, 6]], [['Li', 'As', 'P'], [1, 4, -1], [2, 1, 6]], [['Li', 'P', 'Se'], [1, 4, -1], [2, 1, 6]], [['Li', 'P', 'Br'], [1, 4, -1], [2, 1, 6]], [['Li', 'Sr', 'P'], [1, 4, -1], [2, 1, 6]], [['Li', 'Y', 'P'], [1, 4, -1], [2, 1, 6]], [['Li', 'Zr', 'P'], [1, 4, -1], [2, 1, 6]], [['Li', 'Nb', 'P'], [1, 4, -1], [2, 1, 6]], [['Li', 'Mo', 'P'], [1, 4, -1], [2, 1, 6]], [['Li', 'P', 'Rh'], [1, 4, -1], [2, 1, 6]], [['Li', 'Sn', 'P'], [1, 4, -1], [2, 1, 6]], [['Li', 'Sb', 'P'], [1, 4, -1], [2, 1, 6]], [['Li', 'Te', 'P'], [1, 4, -1], [2, 1, 6]]]
66709


In [293]:
# Using GPU, only finding exact matches - version A

import time
import torch
import copy

def anion_radii_search_a(tol_oct, shannon_data):
    start = time.process_time()
    start_2 = time.time()

    exact_anion_list = []
    oct_list = copy.deepcopy(tol_oct) 

    # Convert numerical parts to GPU tensors
    anion_values = torch.tensor([i[1][2] for i in tol_oct], device="cuda")  # Oxidation state for anion
    cation_values = [f"{i[0][2]}" for i in tol_oct]  # Chemical symbol for anion

    # Boolean mask for negative anion values
    negative_mask = anion_values < 0

    # Apply filtering using GPU operations
    valid_mask = torch.zeros_like(negative_mask, dtype=torch.bool)

    for idx, cat in enumerate(cation_values):
        if cat in shannon_data:
            possible_anions = set(shannon_data[cat].keys())  # Convert to set for faster lookup
            anion = f"{anion_values[idx].item()}"

            if anion in possible_anions:
                valid_mask[idx] = True

    # Extract valid structures using the mask
    exact_anion_list = [tol_oct[i] for i in range(len(tol_oct)) if negative_mask[i] and valid_mask[i]]
    oct_list = [tol_oct[i] for i in range(len(tol_oct)) if not (negative_mask[i] and valid_mask[i])]

    end = time.process_time()
    end_2 = time.time()

    print(f"Exact Anion List Length: {len(exact_anion_list)}")
    print(f"Remaining Oct List Length: {len(oct_list)}")
    print(f"CPU Time: {end - start} seconds")
    print(f"Elapsed Time: {end_2 - start_2} seconds")

    return exact_anion_list, oct_list

# Example usage:
result_anions_a, remaining_oct_list_a = anion_radii_search_a(tol_oct, shannon_data)


Exact Anion List Length: 19853
Remaining Oct List Length: 46856
CPU Time: 4.9091831209999555 seconds
Elapsed Time: 4.9059898853302 seconds


In [294]:
# Using GPU, only finding rough matches - version B

import time
import torch

def anion_radii_search_b(tol_oct, shannon_data):
    start = time.process_time()
    start_2 = time.time()

    exact_anion_list = []
    oct_list = copy.deepcopy(tol_oct) 

    # Convert numerical parts to GPU tensors
    anion_values = torch.tensor([i[1][2] for i in tol_oct], device="cuda")  # Extract anion values
    cation_values = [f"{i[0][2]}" for i in tol_oct]  # Cation values (kept as strings for dictionary lookup)

    # Boolean mask for negative anion values
    negative_mask = anion_values < 0

    # Apply filtering using GPU operations
    valid_mask = torch.zeros_like(negative_mask, dtype=torch.bool)

    for idx, cat in enumerate(cation_values):
        if cat in shannon_data:
            possible_anions = set(shannon_data[cat].keys())  # Convert to set for faster lookup
            anion_plus1 = f"{anion_values[idx].item() + 1}"
            anion_minus1 = f"{anion_values[idx].item() - 1}"

            if anion_plus1 in possible_anions or anion_minus1 in possible_anions:
                valid_mask[idx] = True

    # Extract valid structures using the mask
    exact_anion_list = [tol_oct[i] for i in range(len(tol_oct)) if negative_mask[i] and valid_mask[i]]
    oct_list = [tol_oct[i] for i in range(len(tol_oct)) if not (negative_mask[i] and valid_mask[i])]

    end = time.process_time()
    end_2 = time.time()

    print(f"Exact Anion List Length: {len(exact_anion_list)}")
    print(f"Remaining Oct List Length: {len(oct_list)}")
    print(f"CPU Time: {end - start} seconds")
    print(f"Elapsed Time: {end_2 - start_2} seconds")

    return exact_anion_list, oct_list

# Example usage:
result_anions_b, remaining_oct_list_b = anion_radii_search_b(remaining_oct_list_a, shannon_data)


Exact Anion List Length: 13825
Remaining Oct List Length: 33031
CPU Time: 4.169221557000014 seconds
Elapsed Time: 4.165971755981445 seconds


Screens the number of coumpounds where there is a radius at said oxidation state for the anion

In [296]:
# Checking if the compounds with exact matches for anions an exact match for their cations
# Using GPU, only finding exact matches - version A

import time
import torch

def cation_radii_search_a(tol_oct, shannon_data):
    start = time.process_time()
    start_2 = time.time()

    exact_anion_list = []
    oct_list = copy.deepcopy(tol_oct) 

    # Convert numerical parts to GPU tensors
    a_oxidation = torch.tensor([i[1][0] for i in tol_oct], device="cuda")  # Oxidation state for cation A
    a_symbol = [f"{i[0][0]}" for i in tol_oct]  # Chemical symbol for cation A
    b_oxidation = torch.tensor([i[1][1] for i in tol_oct], device="cuda")  # Oxidation state for cation B
    b_symbol = [f"{i[0][1]}" for i in tol_oct]  # Chemical symbol for cation B

    # Boolean mask for negative anion values
    negative_mask = a_oxidation > 0

    # Apply filtering using GPU operations
    valid_mask = torch.zeros_like(negative_mask, dtype=torch.bool)
    valid_mask_2 = torch.zeros_like(negative_mask, dtype=torch.bool)

    for idx, cat in enumerate(a_symbol):
        if cat in shannon_data:
            possible_ox_a = set(shannon_data[cat].keys())  # All possible oxidation states for A
            cation_a = f"{a_oxidation[idx].item()}"

            if cation_a in possible_ox_a:
                valid_mask[idx] = True

                
    for idx, cat in enumerate(b_symbol):
        if cat in shannon_data:
            possible_ox_b = set(shannon_data[cat].keys())  # All possible oxidation states for B
            cation_b = f"{b_oxidation[idx].item()}"

            if cation_b in possible_ox_b:
                valid_mask_2[idx] = True

    # Extract valid structures using the mask
    exact_anion_list = [tol_oct[i] for i in range(len(tol_oct)) if valid_mask[i] and valid_mask_2[i]]
    oct_list = [tol_oct[i] for i in range(len(tol_oct)) if not (valid_mask[i] and valid_mask_2[i])]

    end = time.process_time()
    end_2 = time.time()

    print(f"Exact Anion List Length: {len(exact_anion_list)}")
    print(f"Remaining Oct List Length: {len(oct_list)}")
    print(f"CPU Time: {end - start} seconds")
    print(f"Elapsed Time: {end_2 - start_2} seconds")

    return exact_anion_list, oct_list


In [297]:
# Both cation having an exact match
cat_an_a, remaining_anion_a = cation_radii_search_a(result_anions_a, shannon_data)

Exact Anion List Length: 7781
Remaining Oct List Length: 12072
CPU Time: 1.9876195089998419 seconds
Elapsed Time: 1.9856529235839844 seconds


In [298]:
# Checking for rough matches as well
# Using GPU, combing exact and rough matches - version G

import time
import torch

def cation_radii_search_g(tol_oct, shannon_data):
    start = time.process_time()
    start_2 = time.time()

    exact_anion_list = []
    oct_list = copy.deepcopy(tol_oct) 

    # Convert numerical parts to GPU tensors
    a_oxidation = torch.tensor([i[1][0] for i in tol_oct], device="cuda")  # Oxidation state for cation A
    a_symbol = [f"{i[0][0]}" for i in tol_oct]  # Chemical symbol for cation A
    b_oxidation = torch.tensor([i[1][1] for i in tol_oct], device="cuda")  # Oxidation state for cation B
    b_symbol = [f"{i[0][1]}" for i in tol_oct]  # Chemical symbol for cation B

    # Boolean mask for negative anion values
    negative_mask = a_oxidation > 0

    # Apply filtering using GPU operations
    valid_mask = torch.zeros_like(negative_mask, dtype=torch.bool)
    valid_mask_2 = torch.zeros_like(negative_mask, dtype=torch.bool)

    for idx, cat in enumerate(a_symbol):
        if cat in shannon_data:
            possible_ox = set(shannon_data[cat].keys())  # All possible oxidation states for A
            cation = f"{a_oxidation[idx].item()}"
            cation_plus1 = f"{a_oxidation[idx].item() + 1}"
            cation_minus1 = f"{a_oxidation[idx].item() - 1}"
            
            if cation in possible_ox or cation_minus1 in possible_ox or cation_plus1 in possible_ox:
                valid_mask[idx] = True

                
    for idx, cat in enumerate(b_symbol):
        if cat in shannon_data:
            possible_ox = set(shannon_data[cat].keys())  # All possible oxidation states for B
            cation = f"{b_oxidation[idx].item()}"
            cation_plus1 = f"{b_oxidation[idx].item() + 1}"
            cation_minus1 = f"{b_oxidation[idx].item() - 1}"
            
            if cation in possible_ox or cation_minus1 in possible_ox or cation_plus1 in possible_ox:
                valid_mask_2[idx] = True

    # Extract valid structures using the mask
    exact_anion_list = [tol_oct[i] for i in range(len(tol_oct)) if valid_mask[i] and valid_mask_2[i]]
    oct_list = [tol_oct[i] for i in range(len(tol_oct)) if not (valid_mask[i] and valid_mask_2[i])]

    end = time.process_time()
    end_2 = time.time()

    print(f"Rough match list length: {len(exact_anion_list)}")
    print(f"Remaining compound list length: {len(oct_list)}")
    print(f"CPU Time: {end - start} seconds")
    print(f"Elapsed Time: {end_2 - start_2} seconds")

    return exact_anion_list, oct_list





In [299]:
cat_an_g, remaining_anion_g = cation_radii_search_g(remaining_anion_a, shannon_data)

Rough match list length: 6240
Remaining compound list length: 5832
CPU Time: 1.909029878000183 seconds
Elapsed Time: 1.9067847728729248 seconds


In [301]:
# SCREENING FROM ROUGH MATCHES OF ANIONS

# Both cation having an exact match
cat_rough_a, remaining_rough_anion_a = cation_radii_search_a(result_anions_b, shannon_data)

Exact Anion List Length: 5257
Remaining Oct List Length: 8568
CPU Time: 1.5929882440000256 seconds
Elapsed Time: 1.590616226196289 seconds


In [302]:
# Both cations having either a rough match or an exact match
cat_rough_g, remaining_rough_anion_g = cation_radii_search_g(remaining_rough_anion_a, shannon_data)

Rough match list length: 4342
Remaining compound list length: 4226
CPU Time: 1.3706654039999648 seconds
Elapsed Time: 1.3683464527130127 seconds


In [304]:
def find_closest_integer(lst, target):
    if not lst:
        return None  # Handle empty list case

    closest = lst[0]
    min_diff = abs(target - closest)
    second_closest = None

    for num in lst[1:]:
        diff = abs(target - num)

        if diff < min_diff:
            closest = num
            min_diff = diff
            second_closest = None  # Reset second closest
        
        elif diff == min_diff:
            second_closest = num  # Store second closest

    # If target is exactly halfway, return both numbers as a string
    if second_closest is not None and (closest + second_closest) / 2 == target:
        return (f"{min(closest, second_closest)}", f"{max(closest, second_closest)}"), min_diff

    return closest, min_diff




In [314]:


def radii_extraction(input):
    check_1 = False
    check_2 = False
    a = 0
    product = []
    
    for i in input: 
        # FOR A
        if i[1][0] not in shannon_data[i[0][0]].keys():  # If target oxidation state is not in dataset
            # Find the closest possible oxidation state
            q, w = find_closest_integer(list(map(int,list(shannon_data[f"{i[0][0]}"].keys()))),i[1][0])
            if type(q) == tuple: # If there are two possible oxidation states equidistant from target
                if "12" in shannon_data[f"{i[0][0]}"][f"{q[0]}"].keys():
                    check_1 = True
                
                if "12" in shannon_data[f"{i[0][0]}"][f"{q[1]}"].keys():
                    check_2 = True
    
                if check_1 == True and check_2 == True: # If exact coordination available in both, take average
                    a = (shannon_data[f"{i[0][0]}"][f"{q[0]}"]["12"]+shannon_data[f"{i[0][0]}"][f"{q[1]}"]["12"])/2
                elif check_1 == True:
                    a = shannon_data[f"{i[0][0]}"][f"{q[0]}"]["12"]
                elif check_2 == True:
                    a = shannon_data[f"{i[0][0]}"][f"{q[1]}"]["12"]
                else: # If not exact coordination match in either 
                    e, r = find_closest_integer(list(map(int,list(shannon_data[f"{i[0][0]}"][f"{q[0]}"].keys()))),12)
                    t, y = find_closest_integer(list(map(int,list(shannon_data[f"{i[0][0]}"][f"{q[1]}"].keys()))),12)
                    if r<y:
                        if type(e)!= tuple:
                            a = shannon_data[f"{i[0][0]}"][f"{q[0]}"][f"{e}"]
                        if type(e)== tuple:
                            a = (shannon_data[f"{i[0][0]}"][f"{q[0]}"][f"{e[0]}"]+shannon_data[f"{i[0][0]}"][f"{q[0]}"][f"{e[1]}"])/2
                    if r>y:
                        if type(t)!= tuple:
                            a = shannon_data[f"{i[0][0]}"][f"{q[1]}"][f"{t}"]
                        if type(e)== tuple:
                            a = (shannon_data[f"{i[0][0]}"][f"{q[0]}"][f"{t[0]}"]+shannon_data[f"{i[0][0]}"][f"{q[0]}"][f"{t[1]}"])/2
                    if r==y:
                        if type(e)!= tuple and type(t)!= tuple:
                            a = (shannon_data[f"{i[0][0]}"][f"{q[0]}"][f"{e}"]+shannon_data[f"{i[0][0]}"][f"{q[1]}"][f"{t}"])/2
                        if type(e) == tuple:
                            a = shannon_data[f"{i[0][0]}"][f"{q[1]}"][f"{t}"]
                        if type(t) == tuple:
                            a = shannon_data[f"{i[0][0]}"][f"{q[0]}"][f"{e}"]
            if type(q)!= tuple: # Now we have valid oxidation state, test for coordination number
                c_v_a, diff_a = find_closest_integer(list(map(int,list(shannon_data[f"{i[0][0]}"][f"{q}"].keys()))),12)
                if type(c_v_a)==int:
                    a = shannon_data[f"{i[0][0]}"][f"{q}"][str(c_v_a)]
                else:
                    a = (shannon_data[f"{i[0][0]}"][f"{q}"][str(c_v_a[0])]+shannon_data[f"{i[0][0]}"][f"{q}"][str(c_v_a[1])])/2
        else:
            a = shannon_data[f"{i[0][0]}"][f"i[1][0]"]["12"]
      
    
        # FOR B
        check_1 = False
        check_2 = False
        b = 0
        if i[1][1] not in shannon_data[i[0][1]].keys():  # If target oxidation state is not in dataset
            # Find the closest possible oxidation state
            q, w = find_closest_integer(list(map(int,list(shannon_data[f"{i[0][1]}"].keys()))),i[1][1])
            if type(q) == tuple: # If there are two possible oxidation states equidistant from target
                if "6" in shannon_data[f"{i[0][1]}"][f"{q[0]}"].keys():
                    check_1 = True
                
                if "6" in shannon_data[f"{i[0][1]}"][f"{q[1]}"].keys():
                    check_2 = True
    
                if check_1 == True and check_2 == True: # If exact coordination available in both, take average
                    b = (shannon_data[f"{i[0][1]}"][f"{q[0]}"]["6"]+shannon_data[f"{i[0][1]}"][f"{q[1]}"]["6"])/2
                elif check_1 == True:
                    b = shannon_data[f"{i[0][1]}"][f"{q[0]}"]["6"]
                elif check_2 == True:
                    b = shannon_data[f"{i[0][1]}"][f"{q[1]}"]["6"]
                else: # If not exact coordination match in either 
                    e, r = find_closest_integer(list(map(int,list(shannon_data[f"{i[0][1]}"][f"{q[0]}"].keys()))),6)
                    t, y = find_closest_integer(list(map(int,list(shannon_data[f"{i[0][1]}"][f"{q[1]}"].keys()))),6)
                    if r<y:
                        if type(e)!= tuple:
                            b = shannon_data[f"{i[0][1]}"][f"{q[0]}"][f"{e}"]
                        if type(e)== tuple:
                            b = (shannon_data[f"{i[0][1]}"][f"{q[0]}"][f"{e[0]}"]+shannon_data[f"{i[0][1]}"][f"{q[0]}"][f"{e[1]}"])/2
                    if r>y:
                        if type(t)!= tuple:
                            b = shannon_data[f"{i[0][1]}"][f"{q[1]}"][f"{t}"]
                        if type(e)== tuple:
                            b = (shannon_data[f"{i[0][1]}"][f"{q[0]}"][f"{t[0]}"]+shannon_data[f"{i[0][1]}"][f"{q[0]}"][f"{t[1]}"])/2
                    if r==y:
                        if type(e)!= tuple and type(t)!= tuple:
                            b = (shannon_data[f"{i[0][1]}"][f"{q[0]}"][f"{e}"]+shannon_data[f"{i[0][1]}"][f"{q[1]}"][f"{t}"])/2
                        if type(e) == tuple:
                            b = shannon_data[f"{i[0][1]}"][f"{q[1]}"][f"{t}"]
                        if type(t) == tuple:
                            b = shannon_data[f"{i[0][1]}"][f"{q[0]}"][f"{e}"]
            if type(q)!= tuple: # Now we have valid oxidation state, test for coordination number
                c_v_b, diff_b = find_closest_integer(list(map(int,list(shannon_data[f"{i[0][1]}"][f"{q}"].keys()))),6)
                if type(c_v_b)==int:
                    b = shannon_data[f"{i[0][1]}"][f"{q}"][str(c_v_b)]
                else:
                    b = (shannon_data[f"{i[0][1]}"][f"{q}"][str(c_v_b[0])]+shannon_data[f"{i[0][1]}"][f"{q}"][str(c_v_b[1])])/2
        else:
            b = shannon_data[f"{i[0][1]}"][f"{i[1][1]}"]["6"]
    
    
        # FOR C
        check_1 = False
        check_2 = False
        c = 0
        if i[1][2] not in shannon_data[i[0][2]].keys():  # If target oxidation state is not in dataset
            # Find the closest possible oxidation state
            q, w = find_closest_integer(list(map(int,list(shannon_data[f"{i[0][2]}"].keys()))),i[1][2])
            if type(q) == tuple: # If there are two possible oxidation states equidistant from target
                if "5" in shannon_data[f"{i[0][2]}"][f"{q[0]}"].keys():
                    check_1 = True
                
                if "5" in shannon_data[f"{i[0][2]}"][f"{q[1]}"].keys():
                    check_2 = True
    
                if check_1 == True and check_2 == True: # If exact coordination available in both, take average
                    c = (shannon_data[f"{i[0][2]}"][f"{q[0]}"]["5"]+shannon_data[f"{i[0][2]}"][f"{q[1]}"]["5"])/2
                elif check_1 == True:
                    c = shannon_data[f"{i[0][2]}"][f"{q[0]}"]["5"]
                elif check_2 == True:
                    c = shannon_data[f"{i[0][2]}"][f"{q[1]}"]["5"]
                else: # If not exact coordination match in either 
                    e, r = find_closest_integer(list(map(int,list(shannon_data[f"{i[0][2]}"][f"{q[0]}"].keys()))),5)
                    t, y = find_closest_integer(list(map(int,list(shannon_data[f"{i[0][2]}"][f"{q[1]}"].keys()))),5)
                    if r<y:
                        if type(e)!= tuple:
                            c = shannon_data[f"{i[0][2]}"][f"{q[0]}"][f"{e}"]
                        if type(e)== tuple:
                            c = (shannon_data[f"{i[0][2]}"][f"{q[0]}"][f"{e[0]}"]+shannon_data[f"{i[0][2]}"][f"{q[0]}"][f"{e[1]}"])/2
                    if r>y:
                        if type(t)!= tuple:
                            c = shannon_data[f"{i[0][2]}"][f"{q[1]}"][f"{t}"]
                        if type(e)== tuple:
                            c = (shannon_data[f"{i[0][2]}"][f"{q[0]}"][f"{t[0]}"]+shannon_data[f"{i[0][2]}"][f"{q[0]}"][f"{t[1]}"])/2
                    if r==y:
                        if type(e)!= tuple and type(t)!= tuple:
                            c = (shannon_data[f"{i[0][2]}"][f"{q[0]}"][f"{e}"]+shannon_data[f"{i[0][2]}"][f"{q[1]}"][f"{t}"])/2
                        if type(e) == tuple:
                            c = shannon_data[f"{i[0][2]}"][f"{q[1]}"][f"{t}"]
                        if type(t) == tuple:
                            c = shannon_data[f"{i[0][2]}"][f"{q[0]}"][f"{e}"]
            if type(q)!= tuple: # Now we have valid oxidation state, test for coordination number
                c_v_c, diff_c = find_closest_integer(list(map(int,list(shannon_data[f"{i[0][2]}"][f"{q}"].keys()))),5)
                if type(c_v_c)==int:
                    c = shannon_data[f"{i[0][2]}"][f"{q}"][str(c_v_c)]
                else:
                    c = (shannon_data[f"{i[0][2]}"][f"{q}"][str(c_v_c[0])]+shannon_data[f"{i[0][2]}"][f"{q}"][str(c_v_c[1])])/2
        else:
            c = shannon_data[f"{i[0][2]}"][f"{i[1][2]}"]["5"]
    

        product.append((i,[a,b,c]))

    
    return product
        
            
            


In [311]:
# Calculate lattice parameter
import math


def lattice_param_calc(full_list):
    ec = 0
    lp_full_list = []
    for i in full_list:
        a = i[1][0]
        b = i[1][1]
        x = i[1][2]
        
        lp_list = []
        try:
            
            lp_1 = (4/math.sqrt(3))*(math.sqrt((a+x)**2 -(2/3)*(b+x)**2)+((b+x)/(math.sqrt(3))))
            lp_2 = (4/math.sqrt(3))*(math.sqrt((a+x)**2 -(4/3)*(x)**2)+(math.sqrt(2/3)*x))
            lp_3 = 2*(b+x)+2*math.sqrt(2)*x
            lp_4 = 4*math.sqrt(2)*x
        except:
            ec += 1

        lp_list.append((lp_1,lp_2,lp_3,lp_4))
        cubic_lattice_parameter = np.average(lp_list)
        prim_volume = (cubic_lattice_parameter**3)/4
        
        lp_full_list.append((i,cubic_lattice_parameter,prim_volume))
    
    return lp_full_list



In [312]:
def flatten(lst):
    result = []
    for item in lst:
        if isinstance(item, (list, tuple)):  # If item is a list or tuple, recurse
            result.extend(flatten(item))
        else:
            result.append(item)
    return result


Perform radii extraction and lattice parameter calculation before exporting data

In [315]:
cat_an_g_with_radii = radii_extraction(cat_an_g)
print(len(cat_an_g_with_radii))
#print(cat_an_g_with_radii[:10])

good_cat_an_g_with_radii = []
for i in cat_an_g_with_radii:
    if (i[1][1]-i[1][0])<0:
        good_cat_an_g_with_radii.append(i)

print(len(good_cat_an_g_with_radii))

6240
4072


In [316]:
# Calculates lattice parameter, flattens list and outputs CSV
lp_full_list = lattice_param_calc(good_cat_an_g_with_radii)  # Change here
flattened_data = [flatten(entry) for entry in lp_full_list]

file_path = f'Phase 2 {len(flattened_data)}.csv'  # Change file path

# Writing to CSV
with open(file_path, mode='w', newline='') as file:
    writer = csv.writer(file)

    # Write the header row
    writer.writerow(["Element Combination", "Oxidation States", "Stoichiometry","Tolerance Factor",
                    "Octahedral factor"])  
    # Write each row (tuple in this case) to the CSV
    for row in flattened_data:   
        writer.writerow(row)

print(f"Data saved to {file_path}")

Data saved to Phase 2 4072.csv
